In [1]:
#!pip install torch 
#!pip install torchsummary
#!pip install torchvision


In [2]:
import torch
from datetime import datetime
import time
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from model_structure import CNN
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
model = CNN().to(device)

In [5]:
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
              ReLU-2         [-1, 32, 224, 224]               0
         MaxPool2d-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 64, 112, 112]          18,496
              ReLU-5         [-1, 64, 112, 112]               0
         MaxPool2d-6           [-1, 64, 56, 56]               0
            Conv2d-7          [-1, 128, 56, 56]          73,856
              ReLU-8          [-1, 128, 56, 56]               0
         MaxPool2d-9          [-1, 128, 28, 28]               0
           Conv2d-10          [-1, 256, 28, 28]         295,168
             ReLU-11          [-1, 256, 28, 28]               0
        MaxPool2d-12          [-1, 256, 14, 14]               0
           Linear-13                  [-1, 512]      25,690,624
             ReLU-14                  [

In [6]:
loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

In [7]:
model_number = 2
# Set data set directory
data_dir = f'RGB_data_for_model_{model_number}/train/'
test_dir = f'RGB_data_for_model_{model_number}/test/'

# Load data set
dataset = datasets.ImageFolder(data_dir,transform=transform)
test_dataset = datasets.ImageFolder(test_dir,transform=transform)

batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    

In [8]:
print(f"Noof classes: {len(dataset.classes)}")
print(f"Classes: {dataset.classes}")
print(f"Total samples: {len(dataset)}")

Noof classes: 2
Classes: ['asquirrel', 'dog']
Total samples: 3000


In [9]:
num_epochs = 50

In [10]:
train_losses = []
test_metrics = {
    'accuracy': [], 'f1': [], 
    'precision': [], 'recall': []
}

for epoch in range(num_epochs):
    start_time = time.time()
    
    # Training Phase
    model.train()
    train_loss = 0.0
    train_preds = []
    train_targets = []
    
    # Progress bar for training
    train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
    
    for inputs, labels in train_pbar:
        inputs = inputs.to(device)
        labels = labels.float().to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_func(outputs.squeeze(), labels)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
        # Store predictions and targets for metrics
        preds = (outputs.squeeze() > 0.5).float().cpu().detach().numpy()
        train_preds.extend(preds)
        train_targets.extend(labels.cpu().numpy())
        
        # Update progress bar
        train_pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    avg_train_loss = train_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # Testing Phase
    model.eval()
    test_loss = 0.0
    test_preds = []
    test_targets = []
    
    # Progress bar for testing
    test_pbar = tqdm(test_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Test]')
    
    with torch.no_grad():
        for inputs, labels in test_pbar:
            inputs = inputs.to(device)
            labels = labels.float().to(device)
            
            outputs = model(inputs)
            loss = loss_func(outputs.squeeze(), labels)
            test_loss += loss.item()
            
            preds = (outputs.squeeze() > 0.5).float().cpu().numpy()
            test_preds.extend(preds)
            test_targets.extend(labels.cpu().numpy())
    
    # Calculate metrics
    test_accuracy = accuracy_score(test_targets, test_preds)
    test_f1 = f1_score(test_targets, test_preds, zero_division=0)
    test_precision = precision_score(test_targets, test_preds, zero_division=0)
    test_recall = recall_score(test_targets, test_preds, zero_division=0)
    
    # Store metrics
    test_metrics['accuracy'].append(test_accuracy)
    test_metrics['f1'].append(test_f1)
    test_metrics['precision'].append(test_precision)
    test_metrics['recall'].append(test_recall)
    
    # Calculate confusion matrix
    cm = confusion_matrix(test_targets, test_preds)
    
    # Calculate time taken for epoch
    epoch_time = time.time() - start_time
    
    # Print detailed metrics
    print(f'\nEpoch {epoch+1}/{num_epochs} - Time: {epoch_time:.2f}s')
    print(f'Train Loss: {avg_train_loss:.4f}')
    print(f'Test Loss: {test_loss/len(test_loader):.4f}')
    print('\nTest Metrics:')
    print(f'Accuracy: {test_accuracy:.4f}')
    print(f'F1 Score: {test_f1:.4f}')
    print(f'Precision: {test_precision:.4f}')
    print(f'Recall: {test_recall:.4f}')
    print('\nConfusion Matrix:')
    print(cm)
    print('-' * 60)
    times = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    if test_f1 == max(test_metrics['f1']):
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': avg_train_loss,
            'test_metrics': test_metrics
        }, f'CNN_model_{model_number}_final_{times}.pth')
    

Epoch 1/50 [Test]: 100%|██████████| 7/7 [00:01<00:00,  3.51it/s]



Epoch 1/50 - Time: 39.74s
Train Loss: 0.6586
Test Loss: 0.5125

Test Metrics:
Accuracy: 0.7300
F1 Score: 0.7245
Precision: 0.7396
Recall: 0.7100

Confusion Matrix:
[[75 25]
 [29 71]]
------------------------------------------------------------


Epoch 2/50 [Test]: 100%|██████████| 7/7 [00:00<00:00,  8.58it/s]



Epoch 2/50 - Time: 19.13s
Train Loss: 0.4949
Test Loss: 0.4044

Test Metrics:
Accuracy: 0.7900
F1 Score: 0.7941
Precision: 0.7788
Recall: 0.8100

Confusion Matrix:
[[77 23]
 [19 81]]
------------------------------------------------------------


Epoch 3/50 [Test]: 100%|██████████| 7/7 [00:00<00:00,  8.03it/s]


...


Epoch 48/50 - Time: 30.64s
Train Loss: 0.0003
Test Loss: 2.5125

Test Metrics:
Accuracy: 0.8650
F1 Score: 0.8670
Precision: 0.8544
Recall: 0.8800

Confusion Matrix:
[[85 15]
 [12 88]]
------------------------------------------------------------


Epoch 49/50 [Test]: 100%|██████████| 7/7 [00:01<00:00,  4.56it/s]



Epoch 49/50 - Time: 28.27s
Train Loss: 0.0003
Test Loss: 1.8356

Test Metrics:
Accuracy: 0.8600
F1 Score: 0.8586
Precision: 0.8673
Recall: 0.8500

Confusion Matrix:
[[87 13]
 [15 85]]
------------------------------------------------------------


Epoch 50/50 [Test]: 100%|██████████| 7/7 [00:01<00:00,  5.39it/s]


Epoch 50/50 - Time: 28.64s
Train Loss: 0.0002
Test Loss: 2.1025

Test Metrics:
Accuracy: 0.8650
F1 Score: 0.8683
Precision: 0.8476
Recall: 0.8900

Confusion Matrix:
[[84 16]
 [11 89]]
------------------------------------------------------------


In [11]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
# Plot training loss
epochs = range(1, len(train_losses) + 1)
ax1.plot(epochs, train_losses, 'b-', label='Training Loss')
ax1.set_title('Training Loss Over Time')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()

# Plot test metrics
for metric in test_metrics:
    ax2.plot(epochs, test_metrics[metric], label=metric.capitalize())
ax2.set_title('Test Metrics Over Time')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Score')
ax2.legend()

plt.tight_layout()
plt.savefig(f'CNN_model_{model_number}_final_metrics.png')
plt.close()